#### Create a model with the duplicated regressor

In [1]:
import tensorflow as tf
from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance
import keras.backend as K
from dlomix.data import load_processed_dataset
from dlomix.models import PrositIntensityPredictor
import os

2024-07-23 15:30:25.145133: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 15:30:25.145183: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 15:30:25.146581: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-23 15:30:25.154438: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-23 15:30:26.604232: W tensorflow/compiler/tf2


Avaliable feature extractors are (use the key of the following dict and pass it to features_to_extract in the Dataset Class):
{
   "atom_count": "Atom count of PTM.",
   "delta_mass": "Delta mass of PTM.",
   "mod_gain": "Gain of atoms due to PTM.",
   "mod_loss": "Loss of atoms due to PTM.",
   "red_smiles": "Reduced SMILES representation of PTM."
}.
When writing your own feature extractor, you can either
    (1) use the FeatureExtractor class or
    (2) write a function that can be mapped to the Hugging Face dataset.
In both cases, you can access the parsed sequence information from the dataset using the following keys, which all provide python lists:
    - _parsed_sequence: parsed sequence
    - _n_term_mods: N-terminal modifications
    - _c_term_mods: C-terminal modifications



In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

print(f'Number of GPUs available: {len(tf.config.list_physical_devices("GPU"))}')
tf.config.set_soft_device_placement(True)

Number of GPUs available: 1


In [3]:
# initialize model
input_mapping = {
        "SEQUENCE_KEY": "modified_sequence",
        "COLLISION_ENERGY_KEY": "collision_energy_aligned_normed",
        "PRECURSOR_CHARGE_KEY": "precursor_charge_onehot",
        "FRAGMENTATION_TYPE_KEY": "method_nbr",
    }

meta_data_keys=["collision_energy_aligned_normed", "precursor_charge_onehot", "method_nbr"]

alphabet = {
    'A': 1,
    'C': 2,
    'D': 3,
    'E': 4,
    'F': 5,
    'G': 6,
    'H': 7,
    'I': 8,
    'K': 9,
    'L': 10,
    'M': 11,
    'N': 12,
    'P': 13,
    'Q': 14,
    'R': 15,
    'S': 16,
    'T': 17,
    'V': 18,
    'W': 19,
    'Y': 20,
    '[]-': 21,
    '-[]': 22,
    '[UNIMOD:737]-': 21,
    'M[UNIMOD:35]': 23,
    'K[UNIMOD:737]': 24,
    'C[UNIMOD:4]': 25,
    '[UNIMOD:1]-': 26,
    }

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model = PrositIntensityPredictor(
    seq_length=30,
    alphabet=alphabet,
    use_prosit_ptm_features=False,
    with_termini=False,
    input_keys=input_mapping,
    meta_data_keys=meta_data_keys
)

model.compile(
optimizer=optimizer,
loss=masked_spectral_distance,
metrics=[masked_pearson_correlation_distance]
)

2024-07-23 15:30:29.590040: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7505 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:03:00.0, compute capability: 6.1


In [4]:
ds = load_processed_dataset('/cmnfs/proj/bmpc_dlomix/datasets/processed/noptm_baseline_small_bs1024_unmod_extended')
ds

DatasetDict({
    train: Dataset({
        features: ['modified_sequence', 'intensities_raw', 'precursor_charge_onehot', 'collision_energy_aligned_normed', 'method_nbr', '_parsed_sequence', '_n_term_mods', '_c_term_mods'],
        num_rows: 7988
    })
    val: Dataset({
        features: ['modified_sequence', 'intensities_raw', 'precursor_charge_onehot', 'collision_energy_aligned_normed', 'method_nbr', '_parsed_sequence', '_n_term_mods', '_c_term_mods'],
        num_rows: 3993
    })
    test: Dataset({
        features: ['modified_sequence', 'intensities_raw', 'precursor_charge_onehot', 'collision_energy_aligned_normed', 'method_nbr', '_parsed_sequence', '_n_term_mods', '_c_term_mods'],
        num_rows: 2000
    })
})

In [5]:
model.fit(
    ds.tensor_test_data,
    validation_data=ds.tensor_test_data,
    epochs=1
)

2024-07-23 15:30:42.311269: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-07-23 15:30:43.632874: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f9c59d6c770 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-23 15:30:43.632924: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1080, Compute Capability 6.1
2024-07-23 15:30:43.646242: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1721748643.825808 2475406 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2/2 [==============================] - 12s 2s/step - loss: 0.8687 - masked_pearson_correlation_distance: 0.8272 - val_loss: 0.7055 - val_masked_pearson_correlation_distance: 0.6148


In [6]:
model.save("/cmnfs/proj/bmpc_dlomix/models/refinement_transfer_learning/random_model.keras")